In [1]:
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm
import time

from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

In [187]:
imputation = True

if imputation == True:
    X = pd.read_csv('./IDA project dataset/Imputed_Dataset.csv').drop(['Label'],axis=1)
else:
    X = pd.read_csv('./IDA project dataset/train_x.csv').drop(['ID'], axis=1)
    
y = pd.read_csv('./IDA project dataset/train_y.csv').drop(['ID'], axis=1)

X['y'] = y

X_nona = pd.get_dummies(X.dropna(), columns=['Loan type', 'Occupation type'])
y = X_nona['y'].to_numpy()
X = X_nona.drop(['y'], axis=1).to_numpy()

print(X.shape, y.shape)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

(76097, 261) (76097,)


In [181]:
X_imp = pd.read_csv('./IDA project dataset/Imputed_Dataset.csv').drop(['Label'],axis=1)
X_imp.head(10)

,Expense,Income,Age,Score1,Score2,Score3,Score4,Score5,Loan type,Occupation type
0,1830.943788,14767.28013,1.0,0.016885,205.196182,22.521523,600.911200,3464.613291,0.0,1.0
1,1645.302546,15272.26775,0.0,0.240375,194.266317,5.349117,600.888816,3374.921455,0.0,1.0
2,1555.026392,17482.49734,0.0,0.213921,183.529871,-1.054954,598.596944,3331.304886,1.0,1.0
3,1681.232638,16257.66493,0.0,0.303909,191.228965,6.971750,602.447203,3392.275849,1.0,1.0
4,1777.648916,16316.29914,1.0,0.300850,224.074728,11.218489,605.947340,3438.864083,0.0,0.0
5,1523.124500,16622.93724,1.0,0.369899,204.834959,-3.645561,602.787598,3315.891612,0.0,1.0
6,1560.817726,15917.47219,0.0,0.104027,169.320992,-3.235722,594.224070,3334.102946,1.0,2.0
7,1713.508753,13528.79379,0.0,0.297326,149.138845,5.000398,597.663724,3407.875016,1.0,2.0
8,1648.118401,14199.98019,0.0,0.118299,190.691595,13.500508,600.088779,3376.281924,0.0,1.0
9,1770.176775,15899.76492,1.0,0.069068,202.016131,8.326076,598.336662,3435.253948,0.0,1.0


In [185]:
X_nona.head(10)

,Expense,Income,Age,Score1,Score2,Score3,Score4,Score5,y,Loan type_A,Loan type_B,Occupation type_X,Occupation type_Y,Occupation type_Z
0,1830.943788,14767.28013,1.0,0.016885,205.196182,22.521523,600.911200,3464.613291,0.0,0,1,0,1,0
1,1645.302546,15272.26775,0.0,0.240375,194.266317,5.349117,600.888816,3374.921455,0.0,0,1,0,1,0
2,1555.026392,17482.49734,0.0,0.213921,183.529871,-1.054954,598.596944,3331.304886,0.0,1,0,0,1,0
5,1523.124500,16622.93724,1.0,0.369899,204.834959,-3.645561,602.787598,3315.891612,0.0,0,1,0,1,0
6,1560.817726,15917.47219,0.0,0.104027,169.320992,-3.235722,594.224070,3334.102946,0.0,1,0,0,0,1
7,1713.508753,13528.79379,0.0,0.297326,149.138845,5.000398,597.663724,3407.875016,0.0,1,0,0,0,1
10,1893.876333,15961.48736,1.0,0.173393,200.266608,7.026415,599.431102,3495.018903,1.0,0,1,0,0,1
11,1657.701614,15312.65398,1.0,0.136250,212.479594,6.174057,600.374454,3380.912016,0.0,0,1,0,1,0
12,1981.609217,14365.93282,0.0,-0.080880,187.157731,32.634985,599.445461,3537.406700,0.0,1,0,1,0,0
14,1436.546259,16523.48807,0.0,0.266537,189.491426,-3.945540,599.717859,3274.061676,0.0,1,0,0,1,0


In [32]:
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
var = pca.explained_variance_ratio_


### Decision Tree 

In [35]:
weight_1 = 1
min_samples_leaf = 11

X_train_pca_1 = X_train_pca[y_train==1]
X_train_pca_0 = X_train_pca[y_train==0]

X_train_norm = np.concatenate((X_train_pca_0,  np.tile(X_train_pca_1, (weight_1, 1))), axis=0)
                             
y_train_norm = np.concatenate((np.zeros(X_train_pca_0.shape[0]), np.ones(weight_1*X_train_pca_1.shape[0])), axis=0)
                              
X_train_norm, y_train_norm = shuffle(X_train_norm, y_train_norm, random_state=0)

In [36]:
clf1 = make_pipeline(StandardScaler(), DecisionTreeClassifier(min_samples_leaf=11))  #
# clf1.fit(X_train_norm, y_train_norm)
# y_val_pred = clf1.predict(X_val_pca)
# val_acc = accuracy_score(y_val, y_val_pred)
# val_f1 = f1_score(y_val, y_val_pred)
# print(val_acc,val_f1)

0.9816002700877785 0.849862258953168


# KNN 

In [168]:
weight_2 = 2
k = 9

X_train_pca_1 = X_train_pca[y_train==1]
X_train_pca_0 = X_train_pca[y_train==0]

X_train_norm = np.concatenate((X_train_pca_0,  np.tile(X_train_pca_1, (weight_2, 1))), axis=0)
                             
y_train_norm = np.concatenate((np.zeros(X_train_pca_0.shape[0]), np.ones(weight_2*X_train_pca_1.shape[0])), axis=0)
                              
X_train_norm, y_train_norm = shuffle(X_train_norm, y_train_norm, random_state=0)

In [38]:
clf2 = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=9, weights='uniform'))  #StandardScaler(),
# clf.fit(X_train_norm, y_train_norm)
# y_val_pred = clf.predict(X_val_pca)

# val_acc = accuracy_score(y_val, y_val_pred)
# val_f1 = f1_score(y_val, y_val_pred)
# print(val_acc,val_f1)

# SVC

In [62]:
weight_3 = 2
C = 10
gamma = 1


X_train_pca_1 = X_train_pca[y_train==1]
X_train_pca_0 = X_train_pca[y_train==0]

X_train_norm = np.concatenate((X_train_pca_0,  np.tile(X_train_pca_1, (weight_3, 1))), axis=0)
                             
y_train_norm = np.concatenate((np.zeros(X_train_pca_0.shape[0]), np.ones(weight_3*X_train_pca_1.shape[0])), axis=0)

X_train_norm, y_train_norm = shuffle(X_train_norm, y_train_norm, random_state=0)

In [37]:
clf3 = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10, gamma=1, probability=True))  #StandardScaler(), 
# clf.fit(X_train_norm, y_train_norm)
# y_val_pred = clf.predict(X_val_pca)


# val_acc = accuracy_score(y_val, y_val_pred)
# val_f1 = f1_score(y_val, y_val_pred)

In [64]:
print(val_acc,val_f1)

0.987677245104659 0.9014844804318489


# Ensembles

In [ ]:
clf_bag1 = BaggingClassifier(base_estimator=clf1,n_estimators=10, random_state=0)

# Boosting

In [147]:
boost  =  AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=12), n_estimators=100,random_state=0)

In [148]:
scaler = StandardScaler()
X_train_trans = scaler.fit_transform(X_train_pca)
X_val_trans = scaler.transform(X_val_pca)

In [149]:
boost.fit(X_train_trans,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=12),
                   n_estimators=100, random_state=0)

In [150]:
y_val_pred = boost.predict(X_val_trans)


val_acc = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
print(val_acc,val_f1)

0.9872552329507089 0.894773519163763


In [151]:
clf4 = make_pipeline(StandardScaler(), AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=12), n_estimators=100,random_state=0))

# Voting

In [165]:
clf_en = VotingClassifier(estimators=[('tree', clf4), ('knn', clf2), ('SVC', clf3)], voting='soft',weights=[1.5,1,2])

In [166]:
weight = 1

X_train_pca_1 = X_train_pca[y_train==1]
X_train_pca_0 = X_train_pca[y_train==0]

X_train_norm = np.concatenate((X_train_pca_0,  np.tile(X_train_pca_1, (weight, 1))), axis=0)
                             
y_train_norm = np.concatenate((np.zeros(X_train_pca_0.shape[0]), np.ones(weight*X_train_pca_1.shape[0])), axis=0)

X_train_norm, y_train_norm = shuffle(X_train_norm, y_train_norm, random_state=0)

In [167]:
clf_en.fit(X_train_pca,y_train)

VotingClassifier(estimators=[('tree',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('adaboostclassifier',
                                               AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=12),
                                                                  n_estimators=100,
                                                                  random_state=0))])),
                             ('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier(n_neighbors=9))])),
                             ('SVC',
                              Pipeline(steps=[('standardscaler',
                                           

In [168]:
y_val_pred = clf_en.predict(X_val_pca)


val_acc = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)
print(val_acc,val_f1)

0.988268062120189 0.9030006978367062


In [160]:
X_train_pca.shape

(47390, 5)